## Set-up

In [ ]:
from google.cloud import bigquery
from google.cloud import storage
from tqdm import tqdm
import matplotlib.pyplot as plt
import ipywidgets
import numpy as np
from google.cloud import aiplatform
import pandas as pd


In [ ]:
!PROJECT_ID=$(gcloud config get-value project)

In [ ]:
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null # to supress any error messages
PROJECT_ID = shell_output[0]
VERTEX_AI_LOCATION = 'europe-west4'

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
aiplatform.init(project = PROJECT_ID, location = VERTEX_AI_LOCATION)

## Overview

There are four ways to export BigQueryMl models:
1. by using the Google Cloud Console,
2. by using `EXPORT MODEL` statement,
3. by using `bq extract` command,
4. Using API or Client Library.

Most of the time the model is saved by default as `TensorfFlow SavedModel`.

## Exporting the model to GCS bucket
By using console and BigQuery

In [ ]:
# list all models
!bq ls -m --format=pretty $PROJECT_ID:BQ_ML_ID

In [ ]:
# Create gcs bucket to store models

bucket_name = "bq-ml-store"
default_storage_class = "STANDARD" 

# Initialize the client
client = storage.Client(project = PROJECT_ID)

# Create the bucket with the specified default storage class
bucket = client.bucket(bucket_name)
bucket.location = "EU"
bucket.storage_class = default_storage_class
    # Try to create the bucket (it will raise an error if it already exists)
try:
    bucket.create()
    print(f"Bucket '{bucket_name}' created with default storage class '{default_storage_class}'.")
except Exception as e:
    print(f"Error creating bucket: {e}")


In [ ]:
!bq extract --model 'BQ_ML_ID.BASE_LOGISTIC_REGRESSION' gs://bq-ml-store/base-logistic-regression

In [ ]:
%%bigquery

 EXPORT MODEL `BQ_ML_ID.DNN`
 OPTIONS(URI = 'gs://bq-ml-store/dnn')
 

## Register the model in Vertex AI

In [ ]:
%%bigquery
ALTER MODEL BQ_ML_ID.BASE_LOGISTIC_REGRESSION SET OPTIONS (vertex_ai_model_id = "base_logistic_regression");

## Deploying Model in Vertex AI

In [ ]:
# create an endpoint

endpoint = aiplatform.Endpoint.create(
        display_name = "base_logistic_regression",
        project = PROJECT_ID,
        location = VERTEX_AI_LOCATION,
    )


In [ ]:
# deploy a model
model = aiplatform.Model(model_name = "base_logistic_regression")
model.deploy(
    endpoint = endpoint,
    deployed_model_display_name = "base_logistic_regression",
    traffic_percentage = 100, # only one model in the endpoint so it must be 100%
    machine_type = "n1-standard-2",
    min_replica_count = 1,
    max_replica_count = 4,
    accelerator_type = None ,
    accelerator_count = None ,
    sync = True,
    )

model.wait()

In [ ]:
!gcloud ai endpoints list --region=europe-west4

In [31]:
endpoint = aiplatform.Endpoint('8224984692508590080')

In [39]:
## Sample prediction
example = {
    "LIMIT_BAL": 20000,
    "SEX": "2",
    "EDUCATION": "2",
    "AGE": 24,

    "PAY_0": "-2",
    "PAY_2": "2",
    "PAY_3": "-1",
    "PAY_4": "-1",
    "PAY_5": "-1",
    "PAY_6": "-1",

    "BILL_AMT1": 0,
    "BILL_AMT2": 0,
    "BILL_AMT3": 0,
    "BILL_AMT4": 0,
    "BILL_AMT5": 0,
    "BILL_AMT6": 0,

    "PAY_AMT1": 0,
    "PAY_AMT2": 0,
    "PAY_AMT3": 0,
    "PAY_AMT4": 0,
    "PAY_AMT5": 0,
    "PAY_AMT6": 0,
}
prediction = endpoint.predict([example])
prediction


Prediction(predictions=[{'card_default_probs': [0.9897584087370261, 0.01024159126297386], 'predicted_card_default': ['1'], 'card_default_values': ['1', '0']}], deployed_model_id='5557499114779836416', model_version_id='1', model_resource_name='projects/115333740492/locations/europe-west4/models/base_logistic_regression', explanations=None)

## Importing a Model to BigQuery ML

The models defined and trained outside of the BigQuery ML can be also imported into the service.
The possible extensions are:
1. XGBoost,
2. Tensorflow,
3. Tensorflow light,
4. Open Neural Network Exchange (ONNX).


### Importing a XGBoost model <br>

I tried importing the XGBoost models but the were many errors especially because at the moment the BigQuery ML does not support the current version of XGBoost, but only below 1.5.1.

### Importing the Tensorflow model 

BigQueryMl should work well with Tensorflow because they are both created by Google. <br>
Keras is a high-level interface for tensorflow which simplifies building the deep-learning models. <br>
AutoKeras is a library that performs automatic machine learning using Keras. <br>
It was developed by Texas A@M University. <br>
Check out the website "https://autokeras.com and the paper https://jmlr.org/papers/v24/20-1355.html .

The training of the model is performed in `keras_model.ipynb`.

In [ ]:
# Coping the model to gcs
# It turns out that you can copy to folder that does not exist and it will be created automatically
!gsutil cp -r auto_keras_classifier/* gs://default-credit-clients-2023/auto_keras_classifier

In [ ]:
%%bigquery

CREATE OR REPLACE MODEL `BQ_ML_ID.AUTO_KERAS_MODEL`
 OPTIONS(MODEL_TYPE='TENSORFLOW',
         MODEL_PATH="gs://default-credit-clients-2023/auto_keras_classifier/*")

In [ ]:
# checking if model was properly imported 
!bq ls -m --format=pretty $PROJECT_ID:BQ_ML_ID